In [31]:
import time,requests,json,random
import pandas as pd
import datetime as dt


In [32]:
class StockCrawling:
    req = None
    db = None

    def __init__(self):
        self.req = requests.session()

    def __del__(self):
        self.req.close()

    def show_realtime(self, *stock_id):
        twse_url = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp'
        timestamp = int(time.time() * 1000)
        channels = '|'.join('tse_{}.tw'.format(target_tse) for target_tse in stock_id)
        query_url = '{}?&ex_ch={}&json=1&delay=0&_={}'.format(twse_url, channels, timestamp)

        headers = {'Accept-Language': 'zh-TW',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
                   }

        self.req.get('http://mis.twse.com.tw/stock/index.jsp', headers=headers)
        response = self.req.get(query_url)
        if response.text.strip() == '':
            return None

        # d: 日期, h: 最高, l: 最低, c:代號, n: 名稱, t:時間, o: 開盤, v: 交易量, z: 成交價
        # load JSON from TSE
        content = json.loads(response.text)
        self.req.cookies.clear()
        return content['msgArray']

In [34]:
if __name__ == '__main__':
    # To ill stock IDs that you want.
    data = pd.read_csv("./StockID/化學類.csv")
    stock_ids = [ str(i) for i in data['代碼'].to_list() ]
    crawling = StockCrawling()
    try:
        while True:
            now = dt.datetime.now()
            #if now < STOP_TIME:
            data = crawling.show_realtime(*stock_ids)
            records = []
            for s_dict in data:
                try:
                    _t = dt.datetime.strptime(s_dict['t'], '%H:%M:%S').time()
                    _dt = dt.datetime.combine(dt.datetime.strptime(s_dict['d'], "%Y%m%d").date(), _t).strftime(
                    '%Y-%m-%d %H:%M:%S')
                    records.append([_dt, s_dict['c'], s_dict['n'],
                                float(s_dict['z']), float(s_dict['h']), float(s_dict['l']), float(s_dict['o']),
                                int(s_dict['v'])])
                except ValueError as _e:
                    print("ValueErr : " + str(_e))
            print(records)
            _sleep_time = random.randint(2, 4)
                # sleep
            time.sleep(_sleep_time)
            print("====== " + time.strftime("%H:%M:%S") + " ======")
    finally:
        del crawling


ValueErr : could not convert string to float: '-'
ValueErr : could not convert string to float: '-'
[['2022-08-10 13:30:00', '1701', '中化', 21.15, 21.25, 20.75, 20.75, 523], ['2022-08-10 13:30:00', '1702', '南僑', 45.75, 45.95, 45.45, 45.45, 184], ['2022-08-10 13:30:00', '1707', '葡萄王', 131.5, 131.5, 130.0, 131.0, 49], ['2022-08-10 13:30:00', '1708', '東鹼', 47.55, 49.2, 47.55, 48.35, 6995], ['2022-08-10 13:30:00', '1709', '和益', 24.9, 24.95, 24.25, 24.25, 2248], ['2022-08-10 13:30:00', '1710', '東聯', 16.85, 17.0, 16.75, 16.75, 475], ['2022-08-10 13:30:00', '1711', '永光', 20.0, 20.15, 19.6, 19.7, 2285], ['2022-08-10 13:30:00', '1712', '興農', 35.5, 35.75, 35.05, 35.05, 2116], ['2022-08-10 13:30:00', '1713', '國化', 23.8, 24.05, 23.5, 23.5, 51], ['2022-08-10 13:30:00', '1714', '和桐', 9.38, 9.45, 9.02, 9.03, 3905], ['2022-08-10 13:30:00', '1717', '長興', 32.2, 32.4, 31.75, 31.85, 1943], ['2022-08-10 13:30:00', '1718', '中纖', 7.8, 7.83, 7.66, 7.66, 1749], ['2022-08-10 13:30:00', '1720', '生達', 45.65, 46.25

KeyboardInterrupt: 